<a href="https://colab.research.google.com/github/saiku122/AIJobcolle/blob/master/MachineLearning/finaltask/LatstTask_ipynb_%E6%94%B9%E8%89%AF(%E3%83%87%E3%83%BC%E3%82%BF%E8%AA%AD%E3%81%BF%E8%BE%BC%E3%81%BF%E5%8F%8A%E3%81%B3%E6%A4%9C%E8%A8%BC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/saiku122/AIJobcolle.git

Cloning into 'AIJobcolle'...
remote: Enumerating objects: 437, done.
remote: Counting objects: 100% (437/437), done.
remote: Compressing objects: 100% (316/316), done.
remote: Total 437 (delta 180), reused 216 (delta 68), pack-reused 0
Receiving objects: 100% (437/437), 12.07 MiB | 16.77 MiB/s, done.
Resolving deltas: 100% (180/180), done.


In [2]:
cd /content/AIJobcolle/MachineLearning/finaltask/

/content/AIJobcolle/MachineLearning/finaltask


★★最終課題★★</br>
〇入力データ</br>
・モデル用：final_hr_analysis_train.csv</br>
・スコア用：final_hr_analysis_test.csv</br>
</br>
〇出力データ</br>
・スコア付データ：</br> 
・モデル：</br>


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import joblib
from sklearn.metrics import accuracy_score
import lightgbm as lgb

In [4]:
#インプットファイル
file_model_name = 'final_hr_analysis_train'
file_test_name = 'final_hr_analysis_test'

In [5]:
my_dtype = {'sales':object,
            'salary':object} ##!!!変更箇所
ohe_cols = ['sales',
           'salary'] #!!!変更箇所

In [6]:
#  モデル用データ読み込み
df = pd.read_csv('./input/'+ file_model_name + '.csv',
                 header=0,
                 dtype=my_dtype)
ID = df.iloc[:,0] #1列目取得
y=df.iloc[:,1]    #2列目の正解データを取得
X=df.iloc[:,2:]   #3列目以降取得
df.dtypes

index                      int64
left                       int64
satisfaction_level       float64
last_evaluation          float64
number_project             int64
average_montly_hours       int64
time_spend_company         int64
Work_accident              int64
promotion_last_5years      int64
sales                     object
salary                    object
dtype: object

In [7]:
# one-hot encoding処理
X_ohe = pd.get_dummies(X, dummy_na=True, columns=ohe_cols)
#display(X_ohe)
X_ohe = X_ohe.dropna(axis=1,how='all')#axis=1　列
#display(X_ohe)
X_ohe_columns = X_ohe.columns.values #モデルデータの列確定

In [8]:
#欠損値の置き換え
imp=SimpleImputer() #平均値で欠損値を補完
imp.fit(X_ohe)
X_ohe = pd.DataFrame(imp.transform(X_ohe),columns=X_ohe_columns) # モデルデータの列を平均値補完適用させ変数を更新

In [9]:
#特徴量抽出
selector = RFECV(estimator=RandomForestClassifier(n_estimators=100,random_state=0), step=0.05) #特徴量をRFE(主成分分析)を用いて次元削減する。(モデルはランダムフォレスト)
selector.fit(X_ohe, y) #RFEした結果を保存
X_ohe_selected = selector.transform(X_ohe) #X_oheに反映
X_ohe_selected = pd.DataFrame(X_ohe_selected, columns=X_ohe_columns[selector.support_]) #削減実行後のデータを再構成

**モデリング**

In [13]:
#アルゴリズム定義
#model_name = 'GBC_001'
'''
clf = Pipeline([('scl',StandardScaler()), 
                ('est',GradientBoostingClassifier(random_state=1))
                ])
'''
# set pipelines for different algorithms
pipelines = {
    'knn':
       Pipeline([('scl',StandardScaler()),
                  ('est',KNeighborsClassifier())]),
    'logistic':
        Pipeline([('scl',StandardScaler()),
                  ('est',LogisticRegression(random_state=1))]),
    'rsvc':
        Pipeline([('scl',StandardScaler()),
                  ('est',SVC(C=1.0, kernel='rbf', class_weight='balanced', random_state=1,probability = True))]),
    'lsvc':
        Pipeline([('scl',StandardScaler()),
                  ('est',SVC(C=1.0, kernel='linear', class_weight='balanced', random_state=1,probability = True))]),
    'tree':
        Pipeline([('scl',StandardScaler()),
                  ('est',DecisionTreeClassifier(random_state=1))]),
    'rf':
        Pipeline([('scl',StandardScaler()),
                  ('est',RandomForestClassifier(random_state=1))]),
    'gb':
        Pipeline([('scl',StandardScaler()),
                  ('est',GradientBoostingClassifier(random_state=1))]),
    'lgb':
        Pipeline([('scl',StandardScaler()),
                  ('lgb',lgb.LGBMClassifier())]),
    'mlp': #ニューラルネットワーク
        Pipeline([('scl',StandardScaler()),
                  ('est',MLPClassifier(hidden_layer_sizes=(3,3),
                                       max_iter=1000,
                                       random_state=1))])
}

**評価データ作成(Holdアウト法)**


In [10]:
#Holdアウト法
from sklearn.model_selection import train_test_split

In [11]:
# holdout法のテストデータ作成
X_train,X_test,y_train,y_test=train_test_split(X_ohe_selected,
                                               y,
                                               test_size=0.3,
                                               random_state=1)

In [16]:
#Holdout法を用いて評価
scores = {}
for pipe_name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)
    scores[(pipe_name,'train')] = accuracy_score(y_train, pipeline.predict(X_train))
    scores[(pipe_name,'test')] = accuracy_score(y_test, pipeline.predict(X_test))

In [17]:
pd.Series(scores).unstack()

,test,train
gb,0.973333,0.976187
knn,0.966667,0.973194
lgb,0.983810,0.996326
logistic,0.766984,0.767860
lsvc,0.745397,0.744727
mlp,0.958095,0.959586
rf,0.987619,0.999864
rsvc,0.962540,0.964485
tree,0.975238,0.999864


**交叉検証**</br>
入力データ：X_ohe_selected</br>
　　　　　　y

In [24]:
from sklearn.model_selection import cross_val_score, KFold
from scipy.stats import sem

In [21]:
#交叉検証評価

In [25]:
scores = {}
cv = KFold(5,shuffle=True,random_state=0)
for pipe_name, pipeline in pipelines.items():
  scores = cross_val_score(pipeline,X_ohe_selected,y,cv=cv)
  print(pipe_name)
  print(scores)
  print ("Mean score: {} (+/-{})".format( np.mean (scores), sem(scores)))

knn
[0.96428571 0.97142857 0.96571429 0.97047619 0.96950929]
Mean score: 0.9682828104085847 (+/-0.0013925709598363188)
logistic
[0.77380952 0.75428571 0.77666667 0.76142857 0.76083849]
Mean score: 0.7654057941423353 (+/-0.004229448100462686)
rsvc
[0.95857143 0.96714286 0.96       0.96571429 0.96379228]
Mean score: 0.9630441706935275 (+/-0.0016395245152398877)
lsvc
[0.74       0.75571429 0.75142857 0.73619048 0.75083373]
Mean score: 0.7468334127362235 (+/-0.003714536022130408)
tree
[0.98       0.97857143 0.97619048 0.97380952 0.97713197]
Mean score: 0.9771406792350099 (+/-0.0010540948033235437)
rf
[0.98904762 0.99       0.99       0.99       0.99047165]
Mean score: 0.9899038544431589 (+/-0.00023273013868065897)
gb
[0.96761905 0.98047619 0.97285714 0.97904762 0.97379705]
Mean score: 0.9747594092424965 (+/-0.002308677707289086)
lgb
[0.98095238 0.98809524 0.9847619  0.98809524 0.98808957]
Mean score: 0.9859988656729962 (+/-0.0014169979671499117)
mlp
[0.95428571 0.96190476 0.9552381  0.9619

In [26]:
#モデル保存
for pipe_name, pipeline in pipelines.items():
  filename=pipe_name + '.joblib'
  joblib.dump('./output/'+ pipe_name, filename)